# Working with Data - Computer Lab for Guest Lecture Julia Lane


In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data.

OUTLINE:
1. Define a research question
2. Think about what data are available
3. Think about possible measurement errors
4. Think about the interpretation of your results
5. Inform your results by linking datasets

# 1. Define a reserach question

Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available

Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset.

In [6]:
#all necessary packages
from __future__ import print_function,division
import re
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gp
import pylab as pl
import json
import urllib2
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from statsmodels import tsa

s = json.load( open(os.getenv('PUIDATA')+'/fbb_matplotlibrc.json') )
pl.rcParams.update(s)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [47]:
#load the data
#url:https://data.cityofnewyork.us/resource/erm2-nwe9.csv
#df=pd.read_csv("https://data.cityofnewyork.us/resource/erm2-nwe9.csv")
url = '/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9'
df = pd.read_csv(url)

In [3]:
#save the data as csv file
df.to_csv("erm2-nwe9", sep='\t', encoding='utf-8')

# 3. Think about possible measurement errors

In [4]:
# Check if all Boroughs and Community Districts are represented in the Data 
df.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

In [5]:
df['Borough'].unique()

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [6]:
# How many unique values do we have? 
print(len(df['Community Board'].unique()))

77


There are 77 community boards but we should only have 59.

In [12]:
print(df['Community Board'].unique())

['13 BROOKLYN' '0 Unspecified' '18 BROOKLYN' '12 BROOKLYN'
 '02 STATEN ISLAND' '08 MANHATTAN' '06 BROOKLYN' '10 BRONX' '06 MANHATTAN'
 '04 BROOKLYN' '04 QUEENS' '09 MANHATTAN' '14 BROOKLYN' '05 MANHATTAN'
 '02 MANHATTAN' '10 QUEENS' '07 QUEENS' '02 QUEENS' '11 BROOKLYN'
 '05 QUEENS' '05 BROOKLYN' '09 QUEENS' '12 MANHATTAN' '01 BROOKLYN'
 '11 MANHATTAN' '03 BRONX' '03 STATEN ISLAND' '08 BROOKLYN' '03 BROOKLYN'
 '07 MANHATTAN' '10 MANHATTAN' '04 BRONX' '01 MANHATTAN' '07 BRONX'
 '09 BRONX' '03 MANHATTAN' '13 QUEENS' '02 BRONX' '12 QUEENS' '16 BROOKLYN'
 '08 QUEENS' '04 MANHATTAN' '14 QUEENS' '10 BROOKLYN' '11 QUEENS'
 '06 BRONX' '08 BRONX' '05 BRONX' '15 BROOKLYN' '02 BROOKLYN' '12 BRONX'
 '03 QUEENS' '06 QUEENS' '01 STATEN ISLAND' '01 BRONX' '01 QUEENS'
 '17 BROOKLYN' '11 BRONX' 'Unspecified QUEENS' '09 BROOKLYN'
 'Unspecified MANHATTAN' 'Unspecified BROOKLYN' '07 BROOKLYN'
 'Unspecified BRONX' '64 MANHATTAN' '81 QUEENS' '55 BROOKLYN' '83 QUEENS'
 'Unspecified STATEN ISLAND' '28 BRONX' 

In [7]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.
print(df['Community Board'].unique())
print(len(re.findall('Unspecified',str(df['Community Board'].unique()))))

['13 BROOKLYN' '0 Unspecified' '18 BROOKLYN' '12 BROOKLYN'
 '02 STATEN ISLAND' '08 MANHATTAN' '06 BROOKLYN' '10 BRONX' '06 MANHATTAN'
 '04 BROOKLYN' '04 QUEENS' '09 MANHATTAN' '14 BROOKLYN' '05 MANHATTAN'
 '02 MANHATTAN' '10 QUEENS' '07 QUEENS' '02 QUEENS' '11 BROOKLYN'
 '05 QUEENS' '05 BROOKLYN' '09 QUEENS' '12 MANHATTAN' '01 BROOKLYN'
 '11 MANHATTAN' '03 BRONX' '03 STATEN ISLAND' '08 BROOKLYN' '03 BROOKLYN'
 '07 MANHATTAN' '10 MANHATTAN' '04 BRONX' '01 MANHATTAN' '07 BRONX'
 '09 BRONX' '03 MANHATTAN' '13 QUEENS' '02 BRONX' '12 QUEENS' '16 BROOKLYN'
 '08 QUEENS' '04 MANHATTAN' '14 QUEENS' '10 BROOKLYN' '11 QUEENS'
 '06 BRONX' '08 BRONX' '05 BRONX' '15 BROOKLYN' '02 BROOKLYN' '12 BRONX'
 '03 QUEENS' '06 QUEENS' '01 STATEN ISLAND' '01 BRONX' '01 QUEENS'
 '17 BROOKLYN' '11 BRONX' 'Unspecified QUEENS' '09 BROOKLYN'
 'Unspecified MANHATTAN' 'Unspecified BROOKLYN' '07 BROOKLYN'
 'Unspecified BRONX' '64 MANHATTAN' '81 QUEENS' '55 BROOKLYN' '83 QUEENS'
 'Unspecified STATEN ISLAND' '28 BRONX' 

Delete unspecified community board, we still have 71 left.

In [8]:
# Check for duplicates? Are these plausible?
print(len(df))
print(len(pd.DataFrame.drop_duplicates(df)))

10187766
10187766


There is no duplicates in the whole data frame. But there might be records that having the same 'Unique key'.

In [9]:
# What about missing values? Can you detect any patterns? 
df.isnull().sum()

Unique Key                               0
Created Date                             0
Closed Date                         450879
Agency                                   0
Agency Name                              0
Complaint Type                           0
Descriptor                           31057
Location Type                      3067596
Incident Zip                        794159
Incident Address                   2197706
Street Name                        2198559
Cross Street 1                     2375943
Cross Street 2                     2425793
Intersection Street 1              8345288
Intersection Street 2              8346302
Address Type                        490299
City                                788712
Landmark                          10180456
Facility Type                      8579485
Status                                   0
Due Date                           7120286
Resolution Description             3786751
Resolution Action Updated Date      250934
Community B

Landmark,School or Citywide Complaint,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name
,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name  
are almost missing for each records.


### Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type to fill missings in Resolution Descriptor?
It is possible to replace missing values. There is no missings in the complaint type so we could use
it to fill missings in resolution description. In our case we could only keep the complaint type.

In [48]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
mask = df['Community Board'].str.contains("Unspecified")

In [49]:
mask.sum()

2064632

In [50]:
# Drop the marked districts
df = df[~mask]

In [53]:
dfnew = df

In [54]:
def check(string):
    flag = True
    if 'Unspecified' in string:
        flag = False
    elif int(string[:2]) > 20:
        flag = False
    return flag
Check = np.vectorize(check)

In [55]:
dfnew["Community Board"] = dfnew["Community Board"][Check(dfnew["Community Board"])]

In [57]:
dfnew['Community Board'].unique()

array(['13 BROOKLYN', '18 BROOKLYN', '12 BROOKLYN', '02 STATEN ISLAND',
       '08 MANHATTAN', '06 BROOKLYN', '10 BRONX', '06 MANHATTAN',
       '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN', '14 BROOKLYN',
       '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS', '07 QUEENS',
       '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN', '09 QUEENS',
       '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN', '03 BRONX',
       '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN', '07 MANHATTAN',
       '10 MANHATTAN', '04 BRONX', '01 MANHATTAN', '07 BRONX', '09 BRONX',
       '03 MANHATTAN', '13 QUEENS', '02 BRONX', '12 QUEENS', '16 BROOKLYN',
       '08 QUEENS', '04 MANHATTAN', '14 QUEENS', '10 BROOKLYN',
       '11 QUEENS', '06 BRONX', '08 BRONX', '05 BRONX', '15 BROOKLYN',
       '02 BROOKLYN', '12 BRONX', '03 QUEENS', '06 QUEENS',
       '01 STATEN ISLAND', '01 BRONX', '01 QUEENS', '17 BROOKLYN',
       '11 BRONX', '09 BROOKLYN', '07 BROOKLYN', nan], dtype=object)

In [59]:
mask = dfnew['Community Board'].isnull()

In [62]:
dfnew=dfnew[~mask]

In [63]:
len(dfnew['Community Board'].unique())

59

Now we have 59 community boards.

In [64]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
reduced_data = dfnew[["Community Board","Complaint Type"]]
rank_complaints = reduced_data.groupby('Community Board').count()
rank_complaints.columns = ['Number of Complaints']
rank_complaints

,Number of Complaints
Community Board,
01 BRONX,74631
01 BROOKLYN,185057
01 MANHATTAN,77974
01 QUEENS,171484
01 STATEN ISLAND,182713
02 BRONX,60257
02 BROOKLYN,121022
02 MANHATTAN,133860
02 QUEENS,114333


In [ ]:
# Safe reduced data frame (Community District level)

## 4. Think about the interpretation of your results?

What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data.

In [7]:
# Population by Community District
df_pop = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv")

In [3]:
#move to PUI data
url = "http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv"
filename = "Final_Demographics.csv"

os.system("curl -O " + url)
print(os.getenv("PUIDATA"))
os.system("mv " + filename + " " + os.getenv("PUIDATA"))

/home/cusp/ss9558/PUIdata


0

In [66]:
# Check variables in file
df_pop.count()

FIPS                                                                                                                                                              59
cd_id                                                                                                                                                             59
Total Population                                                                                                                                                  59
Population Density (per sq. mile)                                                                                                                                 59
% Total Population: Male                                                                                                                                          59
% Total Population: 18 to 24 Years                                                                                                                                59
% Total Po

In [8]:
# How many community districts are in file? 
df_pop["cd_id"].value_counts().sum()

59

In [67]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
df_pop["higher_household"] = df_pop["Households: $125,000 to $149,999"] + \
                             df_pop["Households: $150,000 to $199,999"] + \
                             df_pop["Households: $200,000 or More"]

In [68]:
# Save data frame
df_pop = df_pop[["cd_id","Total Population","higher_household",
                 "Median household income (In 2014 Inflation Adjusted Dollars)"]]

In [69]:
df_pop.head()

,cd_id,Total Population,higher_household,Median household income (In 2014 Inflation Adjusted Dollars)
0,BX08,106737,6000,54224
1,BX12,134644,4761,44906
2,BX10,121209,6857,54962
3,BX11,135839,5163,47910
4,BX03,172247,1428,22041


In [70]:
# Infrastructure by Community District
df_infr = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [9]:
#move to PUI data
url = "http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv"
filename = "ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv"

os.system("curl -O " + url)
print(os.getenv("PUIDATA"))
os.system("mv " + filename + " " + os.getenv("PUIDATA"))

/home/cusp/ss9558/PUIdata


0

In [71]:
# Check variables in file
df_infr.count()

FIPS                                                       59
Geographic Identifier                                      59
Qualifying Name                                            59
Households                                                 59
Households: With An Internet Subscription                  59
Households: Dial-Up Alone                                  59
Households: Dsl                                            59
Households: With Mobile Broadband                          59
Households: Without Mobile Broadband                       59
Households: Cable Modem                                    59
Households: With Mobile Broadband.1                        59
Households: Without Mobile Broadband.1                     59
Households: Fiber-Optic                                    59
Households: With Mobile Broadband.2                        59
Households: Without Mobile Broadband.2                     59
Households: Satellite Internet Service                     59
Househol

In [73]:
# How many community districts are in file? 
df_infr["Qualifying Name"].count()

59

In [74]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr["household_mobile"] = df_infr["Households: With Mobile Broadband"] + \
                              df_infr["Households: With Mobile Broadband.1"] + \
                              df_infr["Households: With Mobile Broadband.2"] + \
                              df_infr["Households: With Mobile Broadband.3"] + \
                              df_infr["Households: With Mobile Broadband.4"] + \
                              df_infr["Households: Mobile Broadband Alone or With Dialup"]

In [75]:
df_infr["household_notmobile"] = df_infr["Households: Without Mobile Broadband"] + \
                                 df_infr["Households: Without Mobile Broadband.1"] + \
                                 df_infr["Households: Without Mobile Broadband.2"] + \
                                 df_infr["Households: Without Mobile Broadband.3"] + \
                                 df_infr["Households: Without Mobile Broadband.4"] + \
                                 df_infr["Households: Internet Access Without A Subscription"] + \
                                 df_infr["Households: No Internet Access"] + \
                                 df_infr["Households: Dial-Up Alone"]

In [76]:
# Aggregate internet type by high and low connections
df_infr["household_highinternet"] = df_infr["Households: Dsl"] + \
                                    df_infr["Households: Cable Modem"] + \
                                    df_infr["Households: Fiber-Optic"] + \
                                    df_infr["Households: Satellite Internet Service"] + \
                                    df_infr["Households: Two or More Fixed Broadband Types, or Other"]

In [77]:
df_infr["household_lowinternet"] = df_infr["Households: Dial-Up Alone"] + \
                                   df_infr["Households: Mobile Broadband Alone or With Dialup"] + \
                                   df_infr["Households: Internet Access Without A Subscription"]

In [78]:
# Save data frame 
df_infr = df_infr[["Qualifying Name","household_mobile","household_notmobile","household_highinternet","household_lowinternet"]]

In [79]:
df_infr.head()

,Qualifying Name,household_mobile,household_notmobile,household_highinternet,household_lowinternet
0,"NYC-Bronx Community District 8--Riverdale, New...",17527,24508,29183,4731
1,"NYC-Bronx Community District 12--Wakefield, Ne...",11414,33416,31137,2997
2,"NYC-Bronx Community District 10--Co-op City, N...",12064,34986,31932,3679
3,NYC-Bronx Community District 11--Pelham Parkwa...,11749,33173,30861,3864
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",16292,41264,34030,4785


# 5. Inform your results by linking datasets

 Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc. https://docs.python.org/2/library/re.html

In [80]:
#reset index of rank_complaints
rank_complaints = rank_complaints.reset_index()

In [81]:
rank_complaints["Community Board"].value_counts()

15 BROOKLYN         1
04 BRONX            1
02 STATEN ISLAND    1
04 MANHATTAN        1
08 MANHATTAN        1
11 MANHATTAN        1
07 MANHATTAN        1
14 QUEENS           1
12 QUEENS           1
02 QUEENS           1
07 BROOKLYN         1
13 QUEENS           1
05 QUEENS           1
07 BRONX            1
03 BRONX            1
12 BRONX            1
01 STATEN ISLAND    1
05 BROOKLYN         1
13 BROOKLYN         1
10 BROOKLYN         1
02 BRONX            1
17 BROOKLYN         1
04 QUEENS           1
10 MANHATTAN        1
08 QUEENS           1
01 BROOKLYN         1
03 QUEENS           1
11 BROOKLYN         1
06 BRONX            1
08 BROOKLYN         1
06 MANHATTAN        1
16 BROOKLYN         1
08 BRONX            1
09 MANHATTAN        1
05 BRONX            1
09 QUEENS           1
01 BRONX            1
09 BROOKLYN         1
02 MANHATTAN        1
07 QUEENS           1
01 QUEENS           1
11 QUEENS           1
18 BROOKLYN         1
10 BRONX            1
03 BROOKLYN         1
09 BRONX  

In [82]:
# Harmonize identifier of dataframe 1
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("MA+.*", "MN")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("ST+.*", "SI")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("BROO+.*", "BK")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("QU+.*", "QN")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("BRON+.*", "BX")

In [83]:
rank_complaints["cd_id"] = rank_complaints["Community Board"].str.extract('([A-Z]{2,})', expand=True)\
                         + rank_complaints["Community Board"].str.extract('([0-9]\d{0,})', expand=True)

In [84]:
rank_complaints

,Community Board,Number of Complaints,cd_id
0,01 BX,74631,BX01
1,01 BK,185057,BK01
2,01 MN,77974,MN01
3,01 QN,171484,QN01
4,01 SI,182713,SI01
5,02 BX,60257,BX02
6,02 BK,121022,BK02
7,02 MN,133860,MN02
8,02 QN,114333,QN02
9,02 SI,121132,SI02


In [85]:
# Harmonize identifier of dataframe 2
df_infr["Qualifying Name"].value_counts()

NYC-Bronx Community District 3 & 6--Belmont, New York                                       2
NYC-Manhattan Community District 4 & 5--Chelsea, New York                                   2
NYC-Bronx Community District 1 & 2--Hunts Point, New York                                   2
NYC-Manhattan Community District 1 & 2--Battery Park City, New York                         2
NYC-Bronx Community District 9--Castle Hill, New York                                       1
NYC-Brooklyn Community District 16--Brownsville & Ocean Hill PUMA, New York                 1
NYC-Brooklyn Community District 8--Crown Heights North & Prospect Heights PUMA, New York    1
NYC-Queens Community District 12--Jamaica, New York                                         1
NYC-Queens Community District 4--Elmhurst & South Corona PUMA, New York                     1
NYC-Brooklyn Community District 4--Bushwick PUMA, New York                                  1
NYC-Brooklyn Community District 11--Bensonhurst & Bath Beach

In [86]:
df_pop.replace('MN11111', 'MN11', inplace=True)
df_pop['cd_id'].unique()
print(len(df_pop["cd_id"]))

59


In [87]:
# Harmonize identifier of dataframe 3
cdid = []
for i in df_infr["Qualifying Name"]:
    cur = i.split(' ') 
    res = [];
    if cur[0] == 'NYC-Manhattan':
        res.append("MN")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Brooklyn':
        res.append("BK")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Bronx':
        res.append("BX")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Staten':
        res.append("SI")
        num = cur[4].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Queens':
        res.append("QN")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    cdid.append(''.join(res))
print (cdid)

['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX03', 'BX07', 'BX05', 'BX04', 'BX09', 'BX01', 'BX01', 'MN12', 'MN09', 'MN10', 'MN11', 'MN08', 'MN07', 'MN04', 'MN04', 'MN06', 'MN03', 'MN01', 'MN01', 'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07', 'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14']


In [88]:
print(len(cdid))
cdid = list(set(cdid))
print(len(cdid))

59
55


In [89]:
#real cdid should be
cdid = ['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05', 'BX04', 'BX09', 'BX01', 'BX02',
       'MA12', 'MA09', 'MA10', 'MA11', 'MA08', 'MA07', 'MA04', 'MA05', 'MA06', 'MA03', 'MA01', 'MA02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18',
       'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14']

In [90]:
df_infr["cd_id"] = cdid

In [91]:
# Link the 3 dataframes
combine = pd.merge(df_infr, rank_complaints, on = "cd_id",how ="inner")
combine = pd.merge(combine, df_pop, on = "cd_id" )
combine

,Qualifying Name,household_mobile,household_notmobile,household_highinternet,household_lowinternet,cd_id,Community Board,Number of Complaints,Total Population,higher_household,Median household income (In 2014 Inflation Adjusted Dollars)
0,"NYC-Bronx Community District 8--Riverdale, New...",17527,24508,29183,4731,BX08,08 BX,92733,106737,6000,54224
1,"NYC-Bronx Community District 12--Wakefield, Ne...",11414,33416,31137,2997,BX12,12 BX,168455,134644,4761,44906
2,"NYC-Bronx Community District 10--Co-op City, N...",12064,34986,31932,3679,BX10,10 BX,95496,121209,6857,54962
3,NYC-Bronx Community District 11--Pelham Parkwa...,11749,33173,30861,3864,BX11,11 BX,122081,135839,5163,47910
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",16292,41264,34030,4785,BX03,03 BX,75134,172247,1428,22041
5,"NYC-Bronx Community District 3 & 6--Belmont, N...",16292,41264,34030,4785,BX06,06 BX,99547,172247,1428,22041
6,"NYC-Bronx Community District 7--Bedford Park, ...",16709,30543,29844,6279,BX07,07 BX,185729,135893,1412,35495
7,NYC-Bronx Community District 5--Morris Heights...,15857,28842,25216,4096,BX05,05 BX,161078,132850,854,20872
8,"NYC-Bronx Community District 4--Concourse, New...",18342,29593,26686,6391,BX04,04 BX,181953,141467,1375,27203
9,"NYC-Bronx Community District 9--Castle Hill, N...",12892,51119,44133,3714,BX09,09 BX,147457,190126,4427,36058


### Are the demographics and infrastructure different in Community Districts that show more complaints than others?
Yes, the demographics and infrastructure different in Community Districts show more complaints than others.
For example,compare BX08 to BX12:BX12 has more population but less household mobiles, but it has more number of complaints than that of BX08.